In [1]:
import pandas as pd

results_file_dir = '/home/roberto.guillen/Documents/multi-nilm/results/complete_results.csv' 
algo_file_dir = '/home/roberto.guillen/Documents/multi-nilm/results/complete_results.csv' 

my_df = pd.read_csv(results_file_dir)
algo_df = pd.read_csv(algo_file_dir)

In [15]:
df.nunique()
print("potato")

5886.40s - Error when getting str with custom provider: <pydevd_plugins.extensions.types.pydevd_plugin_pandas_types.PandasDataFrameTypeResolveProvider object at 0x7f7957b1c490>.
Traceback (most recent call last):
  File "/home/roberto.guillen/.conda/envs/nilmtk-env/lib/python3.7/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_xml.py", line 216, in str_from_providers
    return provider.get_str(o)
  File "/home/roberto.guillen/.conda/envs/nilmtk-env/lib/python3.7/site-packages/debugpy/_vendored/pydevd/pydevd_plugins/extensions/types/pydevd_plugin_pandas_types.py", line 75, in get_str
    show_dimensions=True,
TypeError: to_string() got an unexpected keyword argument 'max_colwidth'


potato


In [4]:
df[['exp_name']]

,exp_name
0,SQLday_dataset_ukdale_components_4_tsLength_10...
1,SQLday_dataset_ukdale_components_4_tsLength_10...
2,SQLday_dataset_ukdale_components_4_tsLength_10...
3,SQLday_dataset_ukdale_components_4_tsLength_10...
4,SQLday_dataset_ukdale_components_4_tsLength_10...
...,...
693,SQLday_dataset_ukdale_components_256_tsLength_...
694,SQLday_dataset_ukdale_components_256_tsLength_...
695,SQLday_dataset_ukdale_components_256_tsLength_...
696,SQLday_dataset_ukdale_components_256_tsLength_...


# Read Other Algorithms

In [27]:
from nilmlab.sql_manager import ResultLogger
import pickle 
from itertools import product
import pandas as pd
import json


result_logger = ResultLogger(db_name='algo')
result_logger.create_db()

db_results = []
empty_exp =  []

print(len(result_logger.get_results()))

for (name, results) in result_logger.get_results():
    if results is not None:        
        df_results = pickle.loads(results)

        df_results.insert(0, 'exp_name', name)

        db_results.append(df_results)
    else: 
        empty_exp.append(name)

complete_results = pd.concat(db_results, axis=0)
complete_results["report"] = complete_results["report"].replace({'\'': '"'}, regex=True)
complete_results["report"] = complete_results["report"].apply(json.loads)

# Remove duplicates
complete_results = complete_results.drop_duplicates(subset=['exp_name'], keep='last')

complete_results.to_csv("algo_complete_results.csv")

28


In [29]:
sample_report = complete_results['report'].iloc[0]

apliances_columns = []

for key in sample_report.keys():

    for metric in sample_report[key]:
        apliances_columns.append(key + "_" + metric)

# Extract a list of all the metrics

report_as_list = complete_results['report'].to_list()

apliances_kpis = []

for report in report_as_list:

    row_metrics = []

    for key in report.keys():

        for metric in report[key]:
            row_metrics.append(report[key][metric])

    apliances_kpis.append(row_metrics)

complete_results = complete_results.reindex(columns=complete_results.columns.tolist() + apliances_columns) 
complete_results[apliances_columns] = apliances_kpis
complete_results.to_csv("algo_complete_results.csv")

In [24]:
empty_exp

['SQLday_dataset_ukdale_algoWAVELETS_tsLength_10Min',
 'SQLday_dataset_ukdale_algoPAA_tsLength_10Min',
 'SQLday_dataset_ukdale_algoPAA_tsLength_Hour',
 'SQLday_dataset_ukdale_algoPAA_tsLength_Day',
 'SQLday_dataset_ukdale_algoSAX1D_tsLength_10Min',
 'SQLday_dataset_ukdale_algoSAX1D_tsLength_Hour',
 'SQLday_dataset_ukdale_algoSAX1D_tsLength_Day',
 'SQLday_dataset_ukdale_algoSAX_tsLength_Day',
 'SQLday_dataset_ukdale_algoWEASEL_tsLength_10Min']

# Rebuild databse

In [ ]:
from nilmlab.sql_manager import ResultLogger
import os 
from itertools import product
import pandas as pd

result_logger = ResultLogger(db_name='nilm')
result_logger.create_db()

datasource_name = "ukdale" 
RESULTS_DIR = "/l/users/roberto.guillen/nilm/results/"

In [ ]:
# If result file exists 
ta = []
for i, (components, ts_length, num_rep_vec,classifier_type) in enumerate(product([2**p for p in range(2, 9)], ["10Min","Hour","Day"], [1,2,4,8],[0,1,2,3])):
    exp_name = "SQLday_dataset_%s_components_%d_tsLength_%s_numRepVec_%d_classifier_type_%d"%(datasource_name, components, ts_length, num_rep_vec,classifier_type) 
    # ta.append(i)
    # continue
    results_file_dir = RESULTS_DIR + datasource_name + "/" 
    results_file_dir = os.path.join(results_file_dir, f'{exp_name}.csv')
    if os.path.exists(results_file_dir): # if exists create db entry
        result_logger.insert_result(exp_name)
        df = pd.read_csv(results_file_dir)
        print("storing",exp_name)
        result_logger.update_result(key=exp_name,value=df)
# print(len(ta))

In [ ]:
result_logger.get_results()

# Read NILM DB

In [ ]:
import pickle
import pandas as pd

db_results = []

for (name, results) in result_logger.get_results():

    if results is not None:        

        df_results = pickle.loads(results)

        df_results.insert(0, 'exp_name', name)

        db_results.append(df_results)

complete_results = pd.concat(db_results, axis=0)
complete_results["report"] = complete_results["report"].replace({'\'': '"'}, regex=True)
complete_results["report"] = complete_results["report"].apply(json.loads)

# Remove duplicates

complete_results = complete_results.drop_duplicates(subset=['exp_name'], keep='last')

complete_results.to_csv("complete_results.csv")

In [30]:
# Convert Report JSon to columns
sample_report = complete_results['report'].iloc[0]

apliances_columns = []

for key in sample_report.keys():

    for metric in sample_report[key]:
        apliances_columns.append(key + "_" + metric)

# Extract a list of all the metrics

report_as_list = complete_results['report'].to_list()

apliances_kpis = []

for report in report_as_list:

    row_metrics = []

    for key in report.keys():

        for metric in report[key]:
            row_metrics.append(report[key][metric])

    apliances_kpis.append(row_metrics)

complete_results = complete_results.reindex(columns=complete_results.columns.tolist() + apliances_columns) 
complete_results[apliances_columns] = apliances_kpis
complete_results.to_csv("complete_results.csv")